In [29]:
import pandas as pd
import numpy as np
import glob
import seaborn as sn
import math
import csv
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torch.optim as optim
import torchvision
from tqdm.notebook import trange, tqdm
from einops import rearrange
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
import copy
from itertools import cycle
import time
import argparse
import os
from sklearn.metrics import accuracy_score
#import tensorboard_logger as tb_logger
import shap
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap
import pickle

In [30]:
with open('shap_val.pkl', 'rb') as file:
    shap_val = pickle.load(file)

with open('wavelength_line.pkl', 'rb') as file:
    wavelength_line = pickle.load(file)

with open('wavelength_cont.pkl', 'rb') as file:
    wavelength_cont = pickle.load(file)

In [31]:
shap_val[0].shape

(59, 1, 2008, 4)

In [32]:
#np.squeeze(shap_val[0], axis=1).shape

In [33]:
def get_mean_shap_val():
    regression_out_length = 143
    all_mean_shap_val = []
    for i in range(0,regression_out_length):
        all_mean_shap_val.append(np.mean(np.squeeze(shap_val[i], axis=1), axis=0))

    # Print the shape of the resulting tensor
    #print(all_mean_shap_val[0].shape)
    #print(all_mean_shap_val[0][:,0].shape)
    return all_mean_shap_val

In [34]:
def get_pos_log():
    np.seterr(divide = 'ignore')
    array_shape = (143, 2008, 4)
    all_shap_val = np.empty(array_shape)
    all_mean_shap_val = get_mean_shap_val()
    #print(all_mean_shap_val[0].shape)
    
    for i, element in enumerate(all_mean_shap_val):
        all_shap_val[i] = element

    #abs_val_log = np.log(np.abs(all_shap_val))
    #positive_values = all_shap_val[all_shap_val > 0]
    positive_vals = np.where(all_shap_val > 0.0, all_shap_val, 0.0)
    positive_val_log = np.where(positive_vals > 0.0, np.log(positive_vals), positive_vals)
    return positive_val_log

In [35]:
def get_neg_log():
    np.seterr(divide = 'ignore')
    array_shape = (143, 2008, 4)
    all_shap_val = np.empty(array_shape)
    all_mean_shap_val = get_mean_shap_val()
    
    for i, element in enumerate(all_mean_shap_val):
        all_shap_val[i] = element

    negative_vals = np.where(all_shap_val < 0.0, all_shap_val, 0.0)
    negative_val_log = np.where(negative_vals < 0.0, np.log(np.abs(negative_vals)), negative_vals)  #np.log(1) = 0, so applying np.log to negative values will not cause any 0 
    return negative_val_log

# Processing Positive Feature Importance Score

In [36]:
def get_top_n_feat_score(feat_ind, total_top_n):
    #feat_ind = 0
    top_n = 1               # to select top 1 value for each lineflux wavelength
    positive_val_log = get_pos_log()
    temp = positive_val_log[:,:,feat_ind]

    #selecting top 1 value for each lineflux wavelength
    for i in range(143):
        sorted_arr = np.sort(temp[i])[::-1]
        # Get the top 10 indices
        top_n_indices = np.argsort(temp[i])[:top_n]
        #print(top_n_indices) 

        other_indices = np.arange(len(temp[i]))
        other_indices = np.delete(other_indices, top_n_indices)
        temp[i][other_indices] = 0.0



    list_of_top_1 = []
    for i in range(143):
        list_of_top_1.append([x for x in temp[i] if x < 0.0][0])

    #selecting top 10 values from all 143 lineflux wavelengths
    #total_top_n = 100         # to select total top 10 values from [143,2008] matrix
    list_of_top_n_indices = np.argsort(list_of_top_1)[:total_top_n]
    #print(list_of_top_n_indices)
    for i in range(143):
        if i not in list_of_top_n_indices:
            temp[i] = 0
        
        else:  #setting values other than zero to -200.0 so that they can be easily identified
            for index, value in enumerate(temp[i]):
                if value != 0.0:
                    print(value, wavelength_line[i], wavelength_cont[index])
                    #temp[i][index] = 200
                    #print(temp[i])
    #print(temp)
    return temp
    #print(cnt)

In [37]:
feat_score = get_top_n_feat_score(0, 10)

-18.385757833112145 1.640420e+03 6.190000e+04
-18.3740465525079 1.666150e+03 1.019000e+05
-18.18990422941624 2.324210e+03 9.830000e+03
-17.92507452958734 4.102892e+03 1.562500e+04
-17.980075907453116 7.137770e+03 1.127000e+07
-17.856782397336474 9.071100e+03 3.020000e+06
-17.716709039206286 1.945089e+04 3.350000e+04
-17.958465991834803 1.436780e+05 1.722000e+06
-19.157622371322677 2.183020e+05 2.421000e+06
-18.6865408053163 3.481400e+05 7.870000e+05


In [38]:
'''
total_top_n_indices = []
for i in range(143):
    sorted_arr = np.sort(feat_score[i])[::-1]
    # Get the top 10 indices
    top_n_indices = np.argsort(feat_score[i])[:1]
    total_top_n_indices.append(top_n_indices[0])
print(total_top_n_indices)
'''

'\ntotal_top_n_indices = []\nfor i in range(143):\n    sorted_arr = np.sort(feat_score[i])[::-1]\n    # Get the top 10 indices\n    top_n_indices = np.argsort(feat_score[i])[:1]\n    total_top_n_indices.append(top_n_indices[0])\nprint(total_top_n_indices)\n'

In [39]:
'''
total_top_n_indices = []
for i in range(143):
    sorted_arr = np.sort(temp[i])[::-1]
    # Get the top 10 indices
    top_n_indices = np.argsort(temp[i])[:top_n]
    total_top_n_indices.append(top_n_indices[0])
print(total_top_n_indices)
'''

'\ntotal_top_n_indices = []\nfor i in range(143):\n    sorted_arr = np.sort(temp[i])[::-1]\n    # Get the top 10 indices\n    top_n_indices = np.argsort(temp[i])[:top_n]\n    total_top_n_indices.append(top_n_indices[0])\nprint(total_top_n_indices)\n'

In [40]:
'''
for i in range(len(total_top_n_indices)):
    if total_top_n_indices[i] > 0:
        temp[i][total_top_n_indices[i]] = - 200.0
'''

'\nfor i in range(len(total_top_n_indices)):\n    if total_top_n_indices[i] > 0:\n        temp[i][total_top_n_indices[i]] = - 200.0\n'

In [50]:
import matplotlib.pyplot as plt
import numpy as np

def plot_stat(feat_ind, tot_top_n):
    
    feat_score = get_top_n_feat_score(feat_ind, tot_top_n)
    fig, ax1 = plt.subplots(figsize=(20, 8))
    
    # Create a colormap for non-zero values
    cmap = plt.get_cmap('viridis')  # Change 'viridis' to the desired colormap for non-zero values
    
    # Create scatter points based on feat_score
    y_coords, x_coords = np.indices(feat_score.shape)
    
    # Set zero values to white
    feat_score_with_white = np.where(feat_score == 0, 1.0, feat_score)  # Map zero values to 1.0 (white)
    
    # Use feat_score_with_white as the color intensity
    scatter = ax1.scatter(x_coords, y_coords, c=feat_score_with_white.flatten(), cmap=cmap, marker='s', s=list(feat_score.flatten()*(-50.0)))
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    
    # Set tick parameters
    ax1.tick_params(axis='both', which='major', labelsize=35, length=0)
    plt.xlabel('Spectral Lines (Contflux)', fontsize=35, fontweight='bold')
    plt.ylabel('Spectral Lines (Lineflux)', fontsize=35, fontweight='bold')

    # Set title based on feat_ind
    titles = [
        'incid luminosity (positive values)',
        'attenuated_incid luminosity (positive values)',
        'diffuse luminosity (positive values)',
        'diffuse continuum luminosity (positive values)'
    ]
    plt.title(titles[feat_ind], fontsize=35, fontweight='bold')

    # Adjust layout and display the scatterplot
    plt.tight_layout()
    plt.show()


In [51]:
_ = widgets.interact(
    plot_stat, 
    feat_ind = list(np.arange(0,4)),
    tot_top_n = list(np.arange(0,100))
)

interactive(children=(Dropdown(description='feat_ind', options=(0, 1, 2, 3), value=0), Dropdown(description='t…

# Processing Negative Feature Importance Score

In [43]:
def get_top_n_neg_feat_score(feat_ind, total_top_n):
    #feat_ind = 0
    top_n = 1               # to select top 1 value for each lineflux wavelength
    negative_vals_log = get_neg_log()
    temp = negative_vals_log[:,:,feat_ind]

    #selecting top 1 value for each lineflux wavelength
    for i in range(143):
        sorted_arr = np.sort(temp[i])[::-1]
        #print(sorted_arr)
        # Select the three largest negative values while ignoring zero
        negative_values = sorted_arr[sorted_arr < 0.0]
        # Get the top n indices
        largest_negative_values = negative_values[:top_n]
        #print(largest_negative_values)
        # Get the indices of the selected values in the original array
        selected_indices = np.where(temp[i] == largest_negative_values[:, None])[1]
        #print(selected_indices)
        other_indices = np.arange(len(temp[i]))
        other_indices = np.delete(other_indices, selected_indices)
        temp[i][other_indices] = 0.0



    list_of_top_1 = []
    for i in range(143):
        list_of_top_1.append([x for x in temp[i] if x < 0.0][0])

    #selecting top 10 values from all 143 lineflux wavelengths
    #total_top_n = 100         # to select total top 10 values from [143,2008] matrix
    list_of_top_n_indices = np.argsort(list_of_top_1)[:total_top_n]
    #print(list_of_top_n_indices)
    for i in range(143):
        if i not in list_of_top_n_indices:
            temp[i] = 0
        
        else:  #setting values other than zero to -200.0 so that they can be easily identified
            for index, value in enumerate(temp[i]):
                if value != 0.0:
                    print(value, wavelength_line[i], wavelength_cont[index])
                    #temp[i][index] = 200
                    #print(temp[i])

    #print(temp)
    return temp
    #print(cnt)

In [48]:
def plot_stat_neg(feat_ind, tot_top_n):
    
    feat_score = get_top_n_neg_feat_score(feat_ind, tot_top_n)
    fig, ax1 = plt.subplots(figsize=(20, 8))
    
    # Create a colormap for non-zero values
    cmap = plt.get_cmap('viridis')  # Change 'viridis' to the desired colormap for non-zero values
    
    # Create scatter points based on feat_score
    y_coords, x_coords = np.indices(feat_score.shape)
    
    # Set zero values to white
    feat_score_with_white = np.where(feat_score == 0, 1.0, feat_score)  # Map zero values to 1.0 (white)
    
    # Use feat_score_with_white as the color intensity
    scatter = ax1.scatter(x_coords, y_coords, c=feat_score_with_white.flatten(), cmap=cmap, marker='s', s=list(feat_score.flatten()*(-300.0)))
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    
    # Set tick parameters
    ax1.tick_params(axis='both', which='major', labelsize=35, length=0)
    plt.xlabel('Spectral Lines (Contflux)', fontsize=35, fontweight='bold')
    plt.ylabel('Spectral Lines (Lineflux)', fontsize=35, fontweight='bold')

    # Set title based on feat_ind
    titles = [
        'incid luminosity (negative values)',
        'attenuated_incid luminosity (negative values)',
        'diffuse luminosity (negative values)',
        'diffuse continuum luminosity (negative values)'
    ]
    plt.title(titles[feat_ind], fontsize=35, fontweight='bold')

    # Adjust layout and display the scatterplot
    plt.tight_layout()
    plt.show()


In [49]:
_ = widgets.interact(
    plot_stat_neg, 
    feat_ind = list(np.arange(0,4)),
    tot_top_n = list(np.arange(0,100))
)

interactive(children=(Dropdown(description='feat_ind', options=(0, 1, 2, 3), value=0), Dropdown(description='t…